## Section Headings Curation and sentences spliter of Chilean Policies

In this notebook we will go through the list of documents i each country to create a dictionary of the first 7 characters of the document code associated to a country.

In [1]:
from pathlib import Path
import boto3, json, operator, os, re, string
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Connection to the AWS S3 bucket
To effectively run this cell you need Omdena's credentials. Please keep them local and do not sync them in GitHub repos nor cloud drives. Before doing anything with this json file, please think of security!!

In [2]:
json_folder = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/credentials/")
# json_folder = Path("C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/credentials/")
filename = "Omdena_key_S3.json"
file = json_folder / filename

with open(file, 'r') as f:
    cred = json.load(f) 

for key in cred:
    KEY = key
    SECRET = cred[key]

s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-2',
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

### Functions

In [ ]:
is_empty = re.compile(r'^\s*\r+\n*|^\s*\.\s*\r+\n*')
def is_empty_line(line):
    return bool(is_empty.findall(line))

is_line_break = re.compile(r'\.\s*$|\.\t*$|;\s*$|:\s*$|\s+y$|,\s*$|\d+\s*$')
def is_end_of_paragraph(line):
    return bool(is_line_break.findall(line))

# Function to calculate the uppercase ratio in a string. It is used to detect section headings
def uppercase_ratio(string):
    if len(re.findall(r'[a-z]',string)) == 0:
        return 1
    else:
        return(len(re.findall(r'[A-Z]',string))/len(re.findall(r'[a-z]',string)))

def end_of_heading(line, flag, content):
    if "URL" in line and "https:" in line:
        flag = False
        content = False
        return flag, content
    else:
        return flag, content

def is_section(line):
    section = False
    for key in section_tags:
        if key in line:
            section = True
            break
    return section
            
def end_of_document(line):
    end_of_file = False
    for key in end_of_file_tags:
        if key in line:
            end_of_file = True
            break
    return end_of_file

# Te function to clear html tags
def clean_html_tags(string):
  return cleanr.sub('', string)
    
def is_por_tanto(line):
    if "POR TANTO" in line:
        return True
    else:
        return False

# Function to remove the last lines of a document, the ones that contain the signatures of the officials. It depends on the
# dictionary "official_positions"
def remove_signatures(line):
    signature = False
    for key in official_positions:
        if key in line:
            signature = True
            break
    return signature

# Function to change accented words by non-accented counterparts. It depends on the dictionary "accent_marks_bugs" 
def remove_accents(string):
    for accent in accents_out.findall(string):
        string = string.replace(accent, accents_dict[accent])
    return string

# Function to merge headlines expressing the same concept in different words. It depends on the dictionary "merges"
def merge_concepts(line):
    for key in merges:
        if key in line:
            line = merges[key]
            break
    return line

def clean_bugs(line):
    for key in bugs:
        if key in line:
            line = line.replace(key, bugs[key])
    return line

def clean_special_characters(line):
    char = clean_special_char.findall(line)
    for item in char:
        for character in item:
            if character != '':
                line = line.replace(character, "")
    return line

def clean_acronyms(line):
    acro = clean_acron.findall(line)
    for item in acro:
        for acronym in item:
            if acronym != '':
                line = line.replace(acronym, clean_punct.sub('', acronym))
    return line

def clean_whitespace(line):
    if whitespaces.sub(' ', line).rstrip().lstrip() != None:
        return whitespaces.sub(' ', line).rstrip().lstrip()
    else:
        return line

decimal_points = re.compile(r'(\b\d+\s*\.\s*\d+|\(\.\)|\b\d+\.\s*[A-Za-z]|\b[A-Za-z]\.\s*\d+)')
def change_decimal_points(line):
    dec = decimal_points.findall(line)
    for decimal in dec:
        if decimal != '':
#             print(decimal)
            line = line.replace(decimal, clean_period.sub('^', decimal))
    return line

def get_decimal_points_back(line):
    line = line.replace("^", ".")
    return line
                
# Function sentence
def clean_sentence(string):
    string = clean_capitulo.sub('', string)
    string = clean_bullet_char.sub('', string).rstrip().lstrip()
    string = clean_bullet_number.sub('', string).rstrip().lstrip()
    string = clean_bullet_point.sub('', string).rstrip().lstrip()
    if string != "":
        return string
    else:
        return ""    
    
# points = re.compile(r'(\b\w+\s*\.\s*\b[^\d\W]+)')
# def check_points(line):
#     return points.findall(line)
#     print(points.findall(line))
# This is to check sentences that have just three words
points = re.compile(r'(\b\w+\b\s*){3,}')
def check_sentence(line):
    if points.findall(line):
        return True
    else:
        return False

def split_into_sentences(line, sep):
    sentence_list = []
    for sentence in line.split(sep):
        if check_sentence(sentence):
            sentence = sentence.rstrip().lstrip()
            sentence_list.append(sentence)
    return sentence_list

# Function to add items to the dictionary with duplicate removal
def add_to_dict(string, dictionary, dupl_dict):
    if string in dupl_dict or string == None:
        pass
    else:
        dupl_dict[string] = 0
        if string in dictionary:
            dictionary[string] = dictionary[string] + 1
        else:
            dictionary[string] = 1
    return dictionary
def full_cleaning(line):
    line = clean_html_tags(line)
    line = remove_accents(line)
    line = clean_special_characters(line)
    line = clean_bugs(line)
    line = clean_acronyms(line)
    line = clean_whitespace(line)
#     print(line)
    line = clean_sentence(line)
#     print(line)
    return line
def is_visto(string, section_name, counter, dictionary):
    if section_name == "VISTO" and len(split_into_sentences(string, ":")) > 1:
        visto = split_into_sentences(string, ":")
        for sentence in split_into_sentences(visto[1], ";"):
            counter += 1
            sentence_id = filename[0:7] + '_' + str(counter)
            dictionary[filename][section]["sentences"][sentence_id] = {"text" : sentence, "labels" : []}
    elif section == "VISTO" and len(split_into_sentences(string, ":")) < 2:
        for sentence in split_into_sentences(string, ";"):
            counter += 1
            sentence_id = filename[0:7] + '_' + str(counter)
            dictionary[filename][section]["sentences"][sentence_id] = {"text" : sentence, "labels" : []}
    return counter, dictionary

### Pipeline to process files from S3 bucket
By executing this cell you will go through all policies in El Salvador and process section headings that will be saved in a dictionary. This should be merged with the notebook that builds up the final json files out of plain txt files.

In [6]:
in_folder = {"Chile/full/" : "Chile", "El Salvador/" : "El Salvador"}
out_folder = "JSON/"
filename = "0185327971f7a647a20ea48ad0d253fa2549ac63" # This is only if you want to test on a single file
# bag_of_words = {}
# sentences = []
# sentences_dict = {}
json_file = {}
for path in in_folder:
    for obj in s3.Bucket('wri-latin-talent').objects.all().filter(Prefix= path):
        if path in obj.key and obj.key.replace(path, "") != "":# and filename in obj.key   # Un comment the previous string to run the code just in one sample document.
            key = obj.key.replace(path,"")
            json_file[key[0:7]] = in_folder[path]
           
       

In [8]:
for item in json_file:
    if "El Salvador" in json_file[item]:
        print(item)

01dcc9c
029d411
02e3658
03030d8
04731ac
060720b
069d9e5
074b95f
07d1de2
07eec23
0817764
081affe
08f5a1a
092e4c7
0c5f5ad
0cc5362
0cffb35
0d090c5
0d471c6
0e2a5ca
0e2ad48
0e9333e
0febf9d
11771b4
12244f1
135664a
161d1f4
1629dfa
17252d8
18d1b46
196c699
19a0f82
1a77612
1b7669f
1c909b8
1cd36a0
1ce9e23
1d19c44
1d5dfdc
1f05fcc
1f15406
1ffdee7
20a2b41
2144417
21dcf30
2235b37
2336314
2369a4b
23f3997
2418c2b
27d89cb
28e15d8
29c84d6
29cb81a
2ab0f3c
2b97317
2c0677d
2ce32ee
2dff019
30220c4
303730b
30ba71c
323ef51
326cdc5
3294386
32ff28d
334f31a
348ca50
34f1f1d
354dd06
35c4d4a
3610a3c
36443d4
369c7dd
36ad0ca
37354e2
37d59e7
38507e7
3c8c7a4
3ce4a28
3eb7dca
3f18af2
3f95071
406a87b
40a1fbd
40c8358
4245aca
42dd77a
43f0373
443a5b6
47963c5
47d5420
485f957
48bdadf
498128e
498aa7d
499d53d
4ae166c
4b50e70
4c4c6e4
4c71364
4ce45b1
4e220b5
4e77902
50f28b4
51a0d9e
5295ca3
545e813
55b3aab
55fdbb4
5705f27
5729835
58a2ada
5a47e49
5bd487f
5c54e9a
5ca494a
5ea6d0c
5eb3387
5f9ee58
6381fac
643a949
644a9bb
644dcc9
6458c88


In [ ]:
out_folder = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Processed/")
filename = "Chile_V2_210109.json"
file = out_folder / filename
with open(file, 'w') as fp:
    json.dump(json_file, fp, indent=4)

In [ ]:
# Loading back the sentences dictionary
out_folder = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Processed/")
filename = "Chile_V2_210109.json"
file = out_folder / filename
with open(file, 'r') as fp:
    json_file = json.load(fp)

In [ ]:
print(len(json_file))
# for k in sorted(sentences_dict):
#     print(k, ":", sentences_dict[k])

#### Short summary

In [ ]:
print("After preprocessing there are {} different headings in El Salvador policies".format(len(bag_of_words)))
print("{} documents have been processed".format(i))
print("There are {} lines of text as sentences".format(len(sentences)))

#### Dictionary items sorted by occurrence

In [ ]:
dict( sorted(bag_of_words.items(), key=operator.itemgetter(1),reverse=True))

#### Dictionary items sorted by heading text

In [ ]:
for k in sorted(bag_of_words):
    print(k, ":", bag_of_words[k])

In [ ]:
sentence_length = {}
for filename in json_file:
    for section in json_file[filename]:
        for sentence_id in json_file[filename][section]["sentences"]:
            length = len(json_file[filename][section]["sentences"][sentence_id]["text"])
            if length in sentence_length:
                sentence_length[length] += 1
            else:
                sentence_length[length] = 1
            if length == 13:
                print(sentence_id, "--", json_file[filename][section]["sentences"][sentence_id])


In [ ]:
sentence_length_list = []
for key, value in sentence_length.items():
    sentence_length_list.append([round(key), round(value)])

In [ ]:
sorted_length = sorted(sentence_length_list,key=lambda x: x[0])
df = pd.DataFrame(sorted_length, columns = ['Sentence_length','Frequency'])

In [ ]:
round(df.describe())

In [ ]:
df[len(df)-20:len(df)]

In [ ]:
x_max = df["Sentence_length"].max()
x_max = 65
x_sep = 1
df[0:x_max].plot.bar(x='Sentence_length', y='Frequency', rot=45, figsize = (18, 10), xticks=range(0, x_max, x_sep), xlim = (0, x_max))

In [ ]:
sorted_arr = np.asarray(sorted_length)
labels, ys = zip(*sorted_arr)
xs = np.arange(len(labels)) 
width = 1

plt.bar(xs, ys, width, align='center')

plt.xticks(xs, labels) #Replace default x-ticks with xs, then replace xs with labels
plt.yticks(ys)

# plt.savefig('netscore.png')

#### Saving sentences as csv

In [ ]:
print(sentences[0:2])

In [ ]:
# path = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/")
path = Path("C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/")
filename = "sentences.npy"
file = path / filename
np_sentences = np.array(sentences)
with open(file, 'wb') as f:
    np.save(f, np_sentences)

### Pipeline to process one file from HD folder
This is a pipeline to process a test file in a local folder.

In [ ]:
path = "C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Documents_de_mostra/Chile/"
# path = "C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Documents_de_mostra/Chile/"
files = os.listdir(path)
print(files[0])

In [ ]:
# path = "C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Documents_de_mostra/Chile/"
path = "C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Documents_de_mostra/Chile/"
data_folder = Path(path)
filename = "00d91bcac2667290e3fb42452f5241be28b031c2"


files = os.listdir(path)
files = ["002c53058e85d383b057fa4cc25a6eb8e7d401e3"]

bag_of_words = {}
json_file = {}

i = 0
for filename in files:
#     if i == 0:
    file_ = data_folder / filename
    with open(file_, 'r', encoding = 'utf-8') as file:
        lines = file.readlines()
#         print("\n", filename, "\n")
        json_file[filename] = {}
        line_counter = 0
        heading_flag = True
        heading_content = False
        has_section = False
        json_file[filename]["HEADING"] = {"tags" : [], "sentences" : {}}
        for line in lines:
            line = clean_whitespace(line)
            # Processing document heading
            if heading_flag:
                if "Tipo Norma" in line:
                    heading_content = True
                if heading_content:
                    line = full_cleaning(line)
                    if line != None:
                        if ":" in line:
                            line_counter += 1
                            sentence_id = filename[0:7] + '_' + str(line_counter)
                            json_file[filename]["HEADING"]["sentences"][sentence_id] = {"text" : line, "labels" : []}
                            json_file[filename]["HEADING"]["sentences"][sentence_id]["text"] = line
                        else:
                            json_file[filename]["HEADING"]["sentences"][sentence_id]["text"] = json_file[filename]["HEADING"]["sentences"][sentence_id]["text"] + " " + line
#                 print("**", line)
                heading_flag, heading_content = end_of_heading(line, heading_flag, heading_content)

            # Breaking when document signatures are found    
            elif end_of_document(line):
#                 print(line)
                break
            # Getting section headings
            elif (uppercase_ratio(line) == 1 and len(line) > 10 and line_counter > 0) or is_section(line):
#                 print("line--", line)
                line = remove_accents(line)
                line = clean_bugs(line)
                line = clean_sentence(line)
                if line == None:
                    continue
                else:
                    has_section = True
                    section = merge_concepts(line)
#                     print("**", section)
                    json_file[filename][section] = {"tags" : [], "sentences" : {}}
#                     bag_of_words = add_to_dict(section, bag_of_words, duplicates_dict)
                    if section == "VISTO" and len(split_into_sentences(line, ":")) > 1:
                        visto = split_into_sentences(line, ":")
                        for sentence in split_into_sentences(visto[1], ";"):
                            line_counter += 1
                            sentence_id = filename[0:7] + '_' + str(line_counter)
                            json_file[filename][section]["sentences"][sentence_id] = {"text" : sentence, "labels" : []}

            elif has_section:
                line = full_cleaning(line)
                if line == None:
                    continue                    
                else:
                    line = change_decimal_points(line)
                    for sentence in split_into_sentences(line, "."):
                        line_counter += 1
                        sentence_id = filename[0:7] + '_' + str(line_counter)
                        json_file[filename][section]["sentences"][sentence_id] = {"text" : sentence, "labels" : []}
        i += 1
    #     data = file.read().replace('\n', '')

In [ ]:
json_file

#### Dictionary items sorted by occurrence

In [ ]:
dict( sorted(bag_of_words.items(), key=operator.itemgetter(1),reverse=True))

#### Dictionary items sorted by heading text

In [ ]:
for k in sorted(bag_of_words):
    print(k, ":", bag_of_words[k])